In [15]:
from pyspark.sql import SparkSession

spark_session = SparkSession.builder\
        .master("spark://192.168.2.97:7077") \
        .appName("Q3")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.cores.max", 2)\
        .getOrCreate()

spark_context = spark_session.sparkContext
        #.config("spark.jars.packages", "uk.co.gresearch.spark:spark-extension_2.12:2.4.0-3.3")\
spark_context.setLogLevel("ERROR")

In [18]:
redComm1 = spark_session.read.json("../RC_2006-03.json")
redComm2 = spark_session.read.json("../RC_2006-04.json")

Py4JJavaError: An error occurred while calling o115.json.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
py4j.ClientServerConn

In [17]:
redComm1.printSchema()

root



In [ ]:
redComm1.take(4)

In [ ]:
subs = redComm1.select('subreddit')
subs.take(5)

In [ ]:
subCountM1 = subs\
    .groupBy("subreddit")\
    .count()\
    .orderBy("count", ascending=False)

In [ ]:
subCountM2 = redComm2\
    .select('subreddit')\
    .groupBy('subreddit')\
    .count()\
    .orderBy('count', ascending=False)

In [ ]:
subCountM1.show()

In [ ]:
subCountM2.join(subCountM1,subCountM2.subreddit ==  subCountM1.subreddit,"left")\
    .show(truncate=False)
subCountM2.join(subCountM1,subCountM2.subreddit ==  subCountM1.subreddit,"leftouter")\
    .show(truncate=False)

In [ ]:
spark_session.stop()